# 一次元CNNによる学習と識別性能評価（提案手法）

---  
  
引数：提案手法でベクトル化したcsvファイルがあるディレクトリまでのパス  
  
---  
  
入力：提案手法でベクトル化したcsvファイル  
  
---
  
出力：ACCURACY[loo]_(ファイル名)_SVM.csv 識別性能評価  
  
---
  
提案手法でベクトル化したデータを1dCNNで学習，交差検証法(leave-one-out交差検証法)を用いて識別性能評価を行う．

In [1]:
import pandas as pd
import sys

# matplotlib inline部分は.pyの時にはコメントアウトしないとエラー出る！

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


コマンドライン引数で提案手法でベクトル化したcsvファイルがあるディレクトリまでのパスを取得

In [3]:

#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../Data_tappingState-2fe_Moter/analysis_by_programs/20171020rn/12ch/RawData/'

# 機械学習するデータ（csvファイルのパラメータ名前）
PM_kizami = ['100', '300']
PM_hole = ['01dim', '012dim']


## ONEdCNN_LOO関数

引数としてTrainingData関数で作成した教師データをX，ラベルをyで受け取る．  
交差検証法の一つleave-one-out交差検証で識別精度評価を行う．

* (1個をテストデータ，残りを教師データにして学習・評価) * すべてのデータ個
* 得られたすべてのデータ個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をTrainingData関数に返す

In [4]:
def ONEdCNN_LOO(X, y):

    # 識別率を格納する配列
    LOOscore = np.zeros(len(X))
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]

    
    # 1個をテストデータ，残りを教師データにして学習・評価
    # すべてのデータに対して行う
    for i in range(len(X)):
        
        print('------ ' + str(i) + ' / ' + str(len(X)) + '回 -----')

        # テストデータ
        X_test = X[i]
        y_test = y[i]

        # テストデータとして使用するデータを除いた教師データを作成
        X_train = np.delete(X, i, 0)
        y_train = np.delete(y, i, 0)


        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test ,2)

        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        LOOscore[i] = score[1]
        

        
    # 評価結果（識別率）の平均を求める
    result = LOOscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print(str(LOOscore) + '\n')

    return result

## TrainingData関数 
引数として読み込みたいTapping/Restのそれぞれのファイル名をfile_tap/file_restで受け取る． 
* 機械学習にかけれるようにデータのベクトル化とラベルを作成
* データとラベルを1dCNN_LOO関数，1dCNN_kCV関数に渡す
* 帰ってきた識別率をまとめてmain関数に返す

In [5]:
def TrainingData(file_rest, file_tap):
    
    # 読み込みたいファイルのパス
    PATH_rest = PATH + file_rest
    PATH_tap = PATH + file_tap

    # csvファイル読み込み
    rest = pd.read_csv(PATH_rest, header = 0)
    tap = pd.read_csv(PATH_tap, header = 0) 
    
    # RestとTappingのデータをまとめる
    all_data = pd.concat([rest, tap], axis = 0)

    # ベクトル化
    X = all_data.as_matrix()

    # ラベル作成
    label_rest = np.zeros(len(rest.index))
    label_tap = np.ones(len(tap.index))

    y = np.r_[label_rest, label_tap]

    
    # 学習とleave-one-out交差検証
    
    print('leave-one-out')

    col_name = 'leave-one-out'
    
    #print(col_name)
    
    # SVM_LOO関数
    result_LOO = ONEdCNN_LOO(X, y)
    
    # 評価結果（識別率）をデータフレームに変換・格納
    ac = pd.DataFrame({col_name : [result_LOO] })
    
    return ac

    

## main関数

In [6]:


if __name__ == '__main__':
    
    
    for kizami in range(len(PM_kizami)):
        
        for hole in range(len(PM_hole)):
            
            PM_dataName = PM_kizami[kizami] + '_' + PM_hole[hole]
        
            # 機械学習するデータ（提案手法でベクトル化したcsvファイル）
            ML_restData = 'TDAvec_autocor_rest_custom_' + PM_dataName + '.csv'
            ML_tapData = 'TDAvec_autocor_tap_custom_' + PM_dataName + '.csv'

            # 出力するデータの行名，ファイル名
            outputIndex = 'TDAvec_autocor(' + PM_kizami[kizami] + ', ' + PM_hole[hole] + ') + 1dCNN'
            outputFile = 'TDAvecAutocor' + PM_hole[hole] + PM_kizami[kizami]


            # 識別率を算出

            print('\n--------- ' + outputIndex + ' Data ---------')
            PS_result = TrainingData(ML_restData, ML_tapData)
            print('\n' + str(PS_result))

            # インデックス名をつける
            result_index = [outputIndex]
            PS_result.index = result_index

            # csv書き出し
            PATH_RESULT = PATH + 'ACCURACY[loo]_' + outputFile + '_1dCNN.csv'
            PS_result.to_csv(PATH_RESULT, index = True)




--------- TDAvec_autocor(100, 01dim) + 1dCNN Data ---------
leave-one-out
------ 0 / 14回 -----
Epoch 1/100
13/13 [==============================] - 0s 33ms/step - loss: 0.4473 - acc: 0.5385
Epoch 2/100
13/13 [==============================] - 0s 388us/step - loss: 0.4340 - acc: 0.5385
Epoch 3/100
13/13 [==============================] - 0s 312us/step - loss: 0.4119 - acc: 0.5385
Epoch 4/100
13/13 [==============================] - 0s 342us/step - loss: 0.3814 - acc: 0.5385
Epoch 5/100
13/13 [==============================] - 0s 379us/step - loss: 0.3455 - acc: 0.5385
Epoch 6/100
13/13 [==============================] - 0s 393us/step - loss: 0.3131 - acc: 0.6154
Epoch 7/100
13/13 [==============================] - 0s 380us/step - loss: 0.3000 - acc: 0.6923
Epoch 8/100
13/13 [==============================] - 0s 369us/step - loss: 0.3106 - acc: 0.5385
Epoch 9/100
13/13 [==============================] - 0s 387us/step - loss: 0.3228 - acc: 0.2308
Epoch 10/100
13/13 [=====================

Epoch 85/100
13/13 [==============================] - 0s 359us/step - loss: 0.1128 - acc: 0.9231
Epoch 86/100
13/13 [==============================] - 0s 454us/step - loss: 0.1113 - acc: 0.9231
Epoch 87/100
13/13 [==============================] - 0s 379us/step - loss: 0.1099 - acc: 0.9231
Epoch 88/100
13/13 [==============================] - 0s 354us/step - loss: 0.1085 - acc: 0.9231
Epoch 89/100
13/13 [==============================] - 0s 406us/step - loss: 0.1072 - acc: 0.9231
Epoch 90/100
13/13 [==============================] - 0s 399us/step - loss: 0.1059 - acc: 0.9231
Epoch 91/100
13/13 [==============================] - 0s 436us/step - loss: 0.1046 - acc: 0.9231
Epoch 92/100
13/13 [==============================] - 0s 399us/step - loss: 0.1032 - acc: 0.9231
Epoch 93/100
13/13 [==============================] - 0s 393us/step - loss: 0.1019 - acc: 0.9231
Epoch 94/100
13/13 [==============================] - 0s 393us/step - loss: 0.1005 - acc: 0.9231
Epoch 95/100
13/13 [==========

Epoch 69/100
13/13 [==============================] - 0s 429us/step - loss: 0.1915 - acc: 0.7692
Epoch 70/100
13/13 [==============================] - 0s 509us/step - loss: 0.1903 - acc: 0.7692
Epoch 71/100
13/13 [==============================] - 0s 392us/step - loss: 0.1892 - acc: 0.7692
Epoch 72/100
13/13 [==============================] - 0s 470us/step - loss: 0.1880 - acc: 0.7692
Epoch 73/100
13/13 [==============================] - 0s 382us/step - loss: 0.1868 - acc: 0.7692
Epoch 74/100
13/13 [==============================] - 0s 353us/step - loss: 0.1856 - acc: 0.7692
Epoch 75/100
13/13 [==============================] - 0s 383us/step - loss: 0.1842 - acc: 0.7692
Epoch 76/100
13/13 [==============================] - 0s 377us/step - loss: 0.1829 - acc: 0.7692
Epoch 77/100
13/13 [==============================] - 0s 383us/step - loss: 0.1816 - acc: 0.7692
Epoch 78/100
13/13 [==============================] - 0s 377us/step - loss: 0.1803 - acc: 0.7692
Epoch 79/100
13/13 [==========

Epoch 53/100
13/13 [==============================] - 0s 372us/step - loss: 0.1778 - acc: 0.6923
Epoch 54/100
13/13 [==============================] - 0s 412us/step - loss: 0.1755 - acc: 0.7692
Epoch 55/100
13/13 [==============================] - 0s 369us/step - loss: 0.1733 - acc: 0.7692
Epoch 56/100
13/13 [==============================] - 0s 422us/step - loss: 0.1712 - acc: 0.7692
Epoch 57/100
13/13 [==============================] - 0s 481us/step - loss: 0.1694 - acc: 0.7692
Epoch 58/100
13/13 [==============================] - 0s 409us/step - loss: 0.1680 - acc: 0.8462
Epoch 59/100
13/13 [==============================] - 0s 346us/step - loss: 0.1666 - acc: 0.8462
Epoch 60/100
13/13 [==============================] - 0s 369us/step - loss: 0.1651 - acc: 0.8462
Epoch 61/100
13/13 [==============================] - 0s 392us/step - loss: 0.1634 - acc: 0.8462
Epoch 62/100
13/13 [==============================] - 0s 357us/step - loss: 0.1614 - acc: 0.8462
Epoch 63/100
13/13 [==========

Epoch 37/100
13/13 [==============================] - 0s 423us/step - loss: 0.2125 - acc: 0.6154
Epoch 38/100
13/13 [==============================] - 0s 388us/step - loss: 0.2050 - acc: 0.6923
Epoch 39/100
13/13 [==============================] - 0s 362us/step - loss: 0.2006 - acc: 0.7692
Epoch 40/100
13/13 [==============================] - 0s 335us/step - loss: 0.1998 - acc: 0.7692
Epoch 41/100
13/13 [==============================] - 0s 337us/step - loss: 0.1971 - acc: 0.7692
Epoch 42/100
13/13 [==============================] - 0s 403us/step - loss: 0.1917 - acc: 0.7692
Epoch 43/100
13/13 [==============================] - 0s 508us/step - loss: 0.1875 - acc: 0.7692
Epoch 44/100
13/13 [==============================] - 0s 435us/step - loss: 0.1863 - acc: 0.6923
Epoch 45/100
13/13 [==============================] - 0s 443us/step - loss: 0.1847 - acc: 0.6923
Epoch 46/100
13/13 [==============================] - 0s 379us/step - loss: 0.1807 - acc: 0.7692
Epoch 47/100
13/13 [==========

...

Epoch 12/100
13/13 [==============================] - 0s 611us/step - loss: 0.2843 - acc: 0.6154
Epoch 13/100
13/13 [==============================] - 0s 571us/step - loss: 0.2596 - acc: 0.5385
Epoch 14/100
13/13 [==============================] - 0s 579us/step - loss: 0.2415 - acc: 0.5385
Epoch 15/100
13/13 [==============================] - 0s 542us/step - loss: 0.2298 - acc: 0.6154
Epoch 16/100
13/13 [==============================] - 0s 552us/step - loss: 0.2244 - acc: 0.6923
Epoch 17/100
13/13 [==============================] - 0s 564us/step - loss: 0.2245 - acc: 0.6154
Epoch 18/100
13/13 [==============================] - 0s 570us/step - loss: 0.2283 - acc: 0.6154
Epoch 19/100
13/13 [==============================] - 0s 539us/step - loss: 0.2333 - acc: 0.6154
Epoch 20/100
13/13 [==============================] - 0s 581us/step - loss: 0.2379 - acc: 0.5385
Epoch 21/100
13/13 [==============================] - 0s 566us/step - loss: 0.2417 - acc: 0.5385
Epoch 22/100
13/13 [==========

Epoch 97/100
13/13 [==============================] - 0s 646us/step - loss: 0.0941 - acc: 1.0000
Epoch 98/100
13/13 [==============================] - 0s 614us/step - loss: 0.0920 - acc: 1.0000
Epoch 99/100
13/13 [==============================] - 0s 641us/step - loss: 0.0898 - acc: 1.0000
Epoch 100/100
1/1 [==============================] - 1s 788ms/step
------ 11 / 14回 -----
Epoch 1/100
13/13 [==============================] - 2s 137ms/step - loss: 0.4919 - acc: 0.4615
Epoch 2/100
13/13 [==============================] - 0s 576us/step - loss: 0.4272 - acc: 0.4615
Epoch 3/100
13/13 [==============================] - 0s 570us/step - loss: 0.3716 - acc: 0.5385
Epoch 4/100
13/13 [==============================] - 0s 552us/step - loss: 0.3203 - acc: 0.5385
Epoch 5/100
13/13 [==============================] - 0s 536us/step - loss: 0.2736 - acc: 0.6154
Epoch 6/100
13/13 [==============================] - 0s 605us/step - loss: 0.2526 - acc: 0.6923
Epoch 7/100
13/13 [=========================

Epoch 81/100
13/13 [==============================] - 0s 694us/step - loss: 0.1522 - acc: 0.8462
Epoch 82/100
13/13 [==============================] - 0s 582us/step - loss: 0.1499 - acc: 0.8462
Epoch 83/100
13/13 [==============================] - 0s 703us/step - loss: 0.1478 - acc: 0.8462
Epoch 84/100
13/13 [==============================] - 0s 763us/step - loss: 0.1456 - acc: 0.8462
Epoch 85/100
13/13 [==============================] - 0s 876us/step - loss: 0.1433 - acc: 0.8462
Epoch 86/100
13/13 [==============================] - 0s 854us/step - loss: 0.1408 - acc: 0.8462
Epoch 87/100
13/13 [==============================] - 0s 581us/step - loss: 0.1383 - acc: 0.8462
Epoch 88/100
13/13 [==============================] - 0s 633us/step - loss: 0.1359 - acc: 0.8462
Epoch 89/100
13/13 [==============================] - 0s 626us/step - loss: 0.1335 - acc: 0.8462
Epoch 90/100
13/13 [==============================] - 0s 715us/step - loss: 0.1310 - acc: 0.8462
Epoch 91/100
13/13 [==========

Epoch 65/100
13/13 [==============================] - 0s 712us/step - loss: 0.4615 - acc: 0.5385
Epoch 66/100
13/13 [==============================] - 0s 686us/step - loss: 0.4615 - acc: 0.5385
Epoch 67/100
13/13 [==============================] - 0s 715us/step - loss: 0.4615 - acc: 0.5385
Epoch 68/100
13/13 [==============================] - 0s 658us/step - loss: 0.4615 - acc: 0.5385
Epoch 69/100
13/13 [==============================] - 0s 665us/step - loss: 0.4615 - acc: 0.5385
Epoch 70/100
13/13 [==============================] - 0s 634us/step - loss: 0.4615 - acc: 0.5385
Epoch 71/100
13/13 [==============================] - 0s 619us/step - loss: 0.4615 - acc: 0.5385
Epoch 72/100
13/13 [==============================] - 0s 643us/step - loss: 0.4615 - acc: 0.5385
Epoch 73/100
13/13 [==============================] - 0s 618us/step - loss: 0.4615 - acc: 0.5385
Epoch 74/100
13/13 [==============================] - 0s 650us/step - loss: 0.4615 - acc: 0.5385
Epoch 75/100
13/13 [==========

Epoch 49/100
13/13 [==============================] - 0s 673us/step - loss: 0.1979 - acc: 0.6923
Epoch 50/100
13/13 [==============================] - 0s 645us/step - loss: 0.1955 - acc: 0.7692
Epoch 51/100
13/13 [==============================] - 0s 709us/step - loss: 0.1933 - acc: 0.6923
Epoch 52/100
13/13 [==============================] - 0s 592us/step - loss: 0.1913 - acc: 0.7692
Epoch 53/100
13/13 [==============================] - 0s 742us/step - loss: 0.1893 - acc: 0.7692
Epoch 54/100
13/13 [==============================] - 0s 694us/step - loss: 0.1870 - acc: 0.7692
Epoch 55/100
13/13 [==============================] - 0s 662us/step - loss: 0.1847 - acc: 0.7692
Epoch 56/100
13/13 [==============================] - 0s 679us/step - loss: 0.1823 - acc: 0.6923
Epoch 57/100
13/13 [==============================] - 0s 681us/step - loss: 0.1801 - acc: 0.7692
Epoch 58/100
13/13 [==============================] - 0s 771us/step - loss: 0.1781 - acc: 0.7692
Epoch 59/100
13/13 [==========